In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

from object.loss import CrossEntropyLabelSmooth, Entropy
from object import network
import os, sys
import os.path as osp
os.environ['CUDA_VISILE_DEVICES'] = '0'

In [2]:
def op_copy(optimizer):
    for param_group in optimizer.param_groups:
        param_group['lr0'] = param_group['lr']
    return optimizer

def lr_scheduler(optimizer, iter_num, max_iter, gamma=10, power=0.75):
    decay = (1 + gamma * iter_num / max_iter) ** (-power)
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr0'] * decay
        param_group['weight_decay'] = 1e-3
        param_group['momentum'] = 0.9
        param_group['nesterov'] = True
    return optimizer

In [3]:
class ImageList(Dataset):
    def __init__(self, imgs_path, transform, mode='RGB'):
        self.imgs_path = imgs_path
        self.transform = transform
        self.mode = mode
    def __len__(self):
        return len(self.imgs_path)
    def __getitem__(self, idx):
        path, label = self.imgs_path[idx].split(',')
        img = Image.open(path).convert(self.mode)
        return self.transform(img), int(label)
    
def train_transform():
    return transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
    ])

def test_transform():
    return transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
    ])


def load_data(source_path, target_path, bsize):
    dsets = {}
    dloaders = {}
    
    src_txt = open(source_path, 'r').readlines()
    target_txt = open(target_path, 'r').readlines()
    
    dsize = len(src_txt)
    train_size = int(0.9 * dsize)
    
    train_txt, val_txt = torch.utils.data.random_split(src_txt, [train_size, dsize - train_size])
    
    dsets['source_train'] = ImageList(train_txt, transform=train_transform())
    dloaders['source_train'] = DataLoader(dsets['source_train'], batch_size=bsize, shuffle=True, drop_last=False)
    
    dsets['source_val'] = ImageList(val_txt, transform=test_transform())
    dloaders['source_val'] = DataLoader(dsets['source_val'], batch_size=bsize, shuffle=True, drop_last=False)
    
    dsets['target_train'] = ImageList(target_txt, transform=train_transform())
    dloaders['target_train'] = DataLoader(dsets['target_train'], batch_size=bsize*2, shuffle=True, drop_last=False)

    
    dsets['target_test'] = ImageList(target_txt, transform=test_transform())
    dloaders['target_test'] = DataLoader(dsets['target_test'], batch_size=bsize*2, shuffle=True, drop_last=False)
    
    return dsets, dloaders

def cal_acc(loader, netF, netB, netC):
    netF.eval()
    netB.eval()
    netC.eval()
    
    pred, true = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            output = netC(netB(netF(x)))
            pred.append(output.float().cpu())
            true.append(y.float())

    pred, true = torch.cat(pred), torch.cat(true)
    pred = nn.Softmax(dim=1)(pred)
    _, pred = torch.max(pred, 1)
    acc = (torch.squeeze(pred).float() == true).float().mean()
    return acc.item()

def source_train(dloaders):
    netF = network.ResBase(res_name='resnet50').cuda()
    netB = network.feat_bootleneck(type='bn', feature_dim=netF.in_features, bottleneck_dim=256).cuda()
    netC = network.feat_classifier(type='wn', class_num=65, bottleneck_dim=256).cuda()
    
    param_group = []
    learning_rate = 1e-2
    for k, v in netF.named_parameters():
        param_group += [{'params': v, 'lr': learning_rate*0.1}]
    for k, v in netB.named_parameters():
        param_group += [{'params': v, 'lr': learning_rate}]
    for k, v in netC.named_parameters():
        param_group += [{'params': v, 'lr': learning_rate}]   
    optimizer = optim.SGD(param_group)
    optimizer = op_copy(optimizer)
    
    max_iter = 20 * len(dloaders['source_train'])
    
    netF.train()
    netB.train()
    netC.train()
    
    best_acc = 0
    bestF, bestB, bestC = None, None, None
    
    for iter_num in range(max_iter):
        total_loss = 0
        total_length = 0
        for i, (source_x, source_y) in enumerate(dloaders['source_train']):
            lr_scheduler(optimizer, iter_num=iter_num, max_iter=max_iter)
            source_x, source_y = source_x.cuda(), source_y.cuda()
            
            outputs = netC(netB(netF(source_x)))
            loss = CrossEntropyLabelSmooth(num_classes=65, epsilon=0.1)(outputs, source_y)
            
            total_loss += len(source_x)*loss.item()
            total_length += len(source_x)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            print('Step: %02d/%02d, Training Loss: %.4f' % (i+1, len(dloaders['source_train']), total_loss / total_length), end='\r')

        acc_val = cal_acc(dloaders['source_val'], netF, netB, netC)
        netF.train()
        netB.train()
        netC.train()
        print('Iter: %03d/%03d, Valid Acc: %.2f%%' % (iter_num + 1, max_iter, 100*acc_val))
        
        if acc_val > best_acc:
            best_acc = acc_val
            bestF, bestB, bestC = netF, netB, netC 

    torch.save(bestF.state_dict(), './model/OfficeHome/source_F.pt')
    torch.save(bestB.state_dict(), './model/OfficeHome/source_B.pt')
    torch.save(bestC.state_dict(), './model/OfficeHome/source_C.pt')

    return bestF, bestB, bestC
    
def target_train(dloaders, netF, netB, netC):
    netF.train()
    netB.train()
    netC.eval()
    
    for k, v in netC.named_parameters():
        v.requires_grad = False
    
    param_group = []
    learning_rate = 1e-2
    for k, v in netF.named_parameters():
        param_group += [{'params': v, 'lr': learning_rate*0.1}]
    for k, v in netB.named_parameters():
        param_group += [{'params': v, 'lr': learning_rate}]
        
    optimizer = optim.SGD(param_group)
    optimizer = op_copy(optimizer)
    
    max_iter = 20
    
    for iter_num in range(max_iter):
        for i, (target_x, target_y) in enumerate(dloaders['target_train']):
            lr_scheduler(optimizer, iter_num=iter_num, max_iter=max_iter)
            target_x, target_y = target_x.cuda(), target_y.cuda()
            
            features = netB(netF(target_x))
            output = netC(features)
            
            softmax_output = nn.Softmax(dim=1)(output)
            entropy_loss = torch.mean(Entropy(softmax_output))
            
            msoftmax = softmax_output.mean(dim=0)
            entropy_loss -= torch.sum(-msoftmax * torch.log(msoftmax + 1e-5))
            
            optimizer.zero_grad()
            entropy_loss.backward()
            optimizer.step()
            
            print('Iter: %02d, Step: %02d/%02d' % (iter_num+1, i+1, len(dloaders['target_train'])), end='\r')
#         acc = cal_acc(dloaders['target_test'], netF, netB, netC)
#         netF.train()
#         netB.train()
#         print('Iter: %03d/%03d, Valid Acc: %.2f%%' % (iter_num + 1, max_iter, 100*acc))
        
    torch.save(netF.state_dict(), './model/OfficeHome/target_F.pt')
    torch.save(netB.state_dict(), './model/OfficeHome/target_B.pt')
    torch.save(netC.state_dict(), './model/OfficeHome/target_C.pt')
    
    return netF, netB, netC

In [4]:
def gen_path(path, name):
    res = ''
    path = osp.join(path, name)
    for i, sub_forder in enumerate(sorted(os.listdir(path))):
        for file in sorted(os.listdir(osp.join(path, sub_forder))):
            res += osp.join(path, sub_forder, file) + ',%d\n' % (i)
            
    return res

In [5]:
def main():
    names = ['Art', 'Clipart', 'Product', 'RealWorld']
    class_num = 65

    data_forder = '../data/OfficeHome'
    train_bs = 32

    source_path = osp.join(data_forder, names[0] + '.txt')
    target_path = osp.join(data_forder, names[1] + '.txt')

    dsets, dloaders = load_data(source_path, target_path, train_bs)
    
    netF = network.ResBase(res_name='resnet50').cuda()
    netB = network.feat_bootleneck(type='bn', feature_dim=netF.in_features, bottleneck_dim=256).cuda()
    netC = network.feat_classifier(type='wn', class_num=65, bottleneck_dim=256).cuda()
    
    netF.load_state_dict(torch.load('./model/OfficeHome/target_F.pt'))
    netB.load_state_dict(torch.load('./model/OfficeHome/target_B.pt'))
    netC.load_state_dict(torch.load('./model/OfficeHome/target_C.pt'))
    print('Accuracy: %.2f%%' % (100*cal_acc(dloaders['target_test'], netF, netB, netC)))
    
#     netF, netB, netC = target_train(dloaders, netF, netB, netC)

#     netF, netB, netC = source_train(dloaders)

In [6]:
main()

Accuracy: 55.85%
